In [2]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import gensim
from datetime import datetime, date

import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append('/home/muef/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
import scoping.models as sm
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from utils.text import *
from django.db.models import Q, Count, Func, F, Sum, Value, Case, When, IntegerField

from parliament.utils import merge_utterance_paragraphs

# find text for examples

In [3]:
# more than 2 mentions of coal + later than 2017
uts_sample = pm.Utterance.objects.filter(search_matches=99, document__date__gte=date(2017,10,1), speaker__party__name='cducsu')
print(uts_sample.count())

44


In [5]:
# print speeches that mention coal early on

p = re.compile("Kohle")

for ut in uts_sample:
    text = ut.paragraph_texts
    m = p.search(text)
    if m and m.start() < 1000:
        print(ut, ut.speaker, ut.speaker.party, ut.document.date)
        print(m.start())
        print(ut.paragraph_texts[:m.start()+100], "...")

Utterance object (3212624) Dr. Klaus-Peter Schulze CDUCSU 2018-03-23
379
Herr Präsident! Liebe Kolleginnen und Kollegen! Meine sehr verehrten Damen und Herren auf den Zuschauertribünen! Zunächst einmal herzlichen Glückwunsch, Frau Ministerin! Ich freue mich auf eine hoffentlich gute und sachliche Zusammenarbeit, und ich finde es gar nicht schlimm, wenn Sie Mitglied der IG BCE sind. Denn das ja schon mehrfach angesprochene Thema „Wie weiter mit der Kohle?“ und das Bestreben, in der Diskussion zu einem Endpunkt zu kommen, werden es notwendig machen ...
Utterance object (3363455) Dr. Joachim Pfeiffer CDUCSU 2017-11-21
195
Frau Präsidentin! Meine sehr geehrten Damen und Herren! Wie ist die Lage? Der Weltmarkt für konventionelle Stromerzeugung verändert sich rasant. Das ist nicht neu. Wir haben es beim Übergang von Kohle zu Öl erlebt, wir haben es bei der Kernenergie erlebt, wir erleben es bei der Steinkohle, bei  ...
Utterance object (3368431) Dr. Klaus-Peter Schulze CDUCSU 2018-03-23
379


In [14]:
# print speeches with search term

uts_sample = pm.Utterance.objects.filter(search_matches=89,
                                         document__date__gte=date(1998,1,1),
                                         document__date__lte=date(2005,12,31))
print("searching in {} utterances".format(uts_sample.count()))

p = re.compile("coemission")

for ut in uts_sample:
    text = ut.paragraph_texts
    m = p.search(text)
    if m:
        print(ut, ut.speaker, ut.speaker.party, ut.document.date)
        print(ut.paragraph_texts)
        print("")

searching in 886 utterances


In [33]:
# search texts scoring high in a particular topic

run_id = 2350
topic_title = 'energy transition & power market'
topic = Topic.objects.get(title=topic_title, run_id=run_id)

filtered_uts = pm.Utterance.objects.filter(
    doctopic__topic=topic,
    doctopic__run_id=run_id,
    doctopic__score__gt=0.01,
#    speaker__party__name="gruene",
#    document__date__gte=date(1998,1,1),
#    document__date__lte=date(2005,12,31)
)

rank_uts = {ut['id']: i+1 for i, ut in enumerate(pm.Utterance.objects.filter(
    doctopic__topic=topic,
    doctopic__run_id=run_id
).order_by('-doctopic__score').values('id'))}

p = re.compile("m")

counter = 0

for ut in filtered_uts.order_by('-doctopic__score'):
    text = ut.paragraph_texts
    m = p.search(text)
    if m:
        counter += 1
        if counter < 20:
            score = DocTopic.objects.get(run_id=2350, topic=topic, ut=ut).score
            print(ut, "protocol {}/{},".format(ut.document.parlperiod.n, ut.document.sitting),
                  ut.document.date, "Score:", score, "Rank:", rank_uts[ut.id])
            print("{} ({}):".format(ut.speaker, ut.speaker.party))
            print(ut.paragraph_texts)
            print("")
            
print(counter)

Utterance object (2077864) protocol 16/230, 2009-07-02 Score: 575.2251810362059 Rank: 1
Rolf Hempelmann (SPD):
Zum Abschluss der Legislaturperiode diskutieren wir heute noch einmal über eine Reihe von energiepolitischen Anträgen der Opposition unterschiedlichster Couleur aus den vergangenen fünfzehn Monaten.
In den Anträgen von FDP und Grünen aus 2008 werden in unterschiedlicher Ausprägung Maßnahmen zur Verbesserung des Wettbewerbs auf den Energiemärkten gefordert. Hierzu ist zu sagen, dass Koalition und Bundesnetzagentur in den vergangenen Jahren viel für eine Verbesserung des Wettbewerbs auf den Strom- und Gasmärkten erreicht haben: Zwei Entgeltgenehmigungsrunden bei den Strom- und Gasnetzen haben eine Einsparung bei den Netzentgelten von insgesamt rund 4 Milliarden Euro gebracht. Die Anreizregulierung wurde zum 1. Januar 2009 eingeführt. Die Verabschiedung der Kraftwerksnetzanschlussverordnung hat einen erheblichen Fortschritt hin zu einem diskriminierungsfreien Netzanschluss neuer 

Utterance object (2110565) protocol 18/153, 2016-01-29 Score: 294.2226201853926 Rank: 14
Thomas Bareiß (CDUCSU):
Sehr geehrte Frau Präsidentin! Sehr geehrte Damen! Meine Herren! Liebe Frau Bulling-Schröter, so sieht keine in sich konsistente Energiepolitik aus. Hier kämpfen Sie immer für den Braunkohleausstieg, vor Ort sind Sie dagegen. Hier kämpfen Sie für den Netzausbau, vor Ort sind Sie dagegen.
So ist keine konsistente Energiepolitik zu machen. Unsere Energiepolitik ist aus einem Guss und geht auch in die richtige Richtung.
So ist auch das Strommarktpaket, das auf dem Tisch liegt, zu verstehen. Wir sorgen dafür, dass diese Umwälzung, die in den nächsten Jahren vor uns liegt, auch bewerkstelligt wird. Es kam in den Reden schon zum Ausdruck: Wir haben einen enormen Umbruch in unserer Energielandschaft, und wir haben auch schon sehr große Erfolge vorzuweisen. Wir haben einen Anstieg des Anteils der erneuerbaren Energien an der Stromerzeugung, der so gar nicht vorhersehbar war und den 

2259


In [31]:
#pm.Utterance.objects.get(id=3363455).paragraph_texts